Title: Greenwashing Responses with 3 Different Prompts

Overview:
This script takes in original climate disclosure responses and greenwashes them using the prompts below.

Main Features:
- Rewrites company responses to two key CDP questions:
    Absolute Emissions Targets (C4.1a)
    Emissions Intensity Targets (C4.1b)

- Supports three prompt strictness levels:
    Loose: Ignores length/accuracy constraints.
    Medium: Enforces accuracy (factual intent) but not length
    Strict: Preserves accuracy and length (original word count)


How to Run:
1. Install dependencies:
    openai
    pandas
    numpy

2. Set up your OpenAI API key in `api_keys.py`.
    ex. # api_keys.py
        OPEN_AI_API_KEYS = ["your-api-key-here"]

3. Adjust the input and output file paths as needed in the script.


Note:
Make sure the company responses are aligned with the model's input requirements.

prompt to ask chatgpt to rewrite response in order to make it seem like an A List company

In [ ]:
import openai
MODEL = 'gpt-4o-2024-08-06'
from api_keys import OPEN_AI_API_KEYS
key = OPEN_AI_API_KEYS[0]
client = openai.OpenAI(api_key=key)
openai.api_key = key

import numpy as np
import pandas as pd
import csv

In [ ]:
def safe_cast_to_int(s):
    try:
        return int(s)
    except ValueError:
        return None 

In [ ]:
def prepare_messages(prompts):
	messages = []
	for prompt in prompts:
		line = {"role": "user", "content": prompt}
		messages.append(line)
	return messages

def get_model_output_initial_sentences(prompts, client, model):
    completion = client.chat.completions.create(
        model=MODEL,
        messages=prepare_messages(prompts),
        temperature=0
    )
    output = completion.choices[0].message.content
    return output

In [ ]:
Response_A_2022 = "C4.1a_C27_Provide details of your absolute emissions target(s) and progress made against those targets. - Please explain target coverage and identify any exclusions"
Response_B_2022 = "C4.1b_C30_Provide details of your emissions intensity target(s) and progress made against those target(s). - Please explain target coverage and identify any exclusions"

get_prompt_loose: doesnt enforce length or accuracy

In [ ]:
def get_prompt_loose(response_a, response_b):
    prompt_a = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        f"[The Start of Company’s Question 1 Response] {response_a} [The End of Company’s Response]"
    ]
    
    prompt_b = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        f"[The Start of Company’s Question 2 Response] {response_b} [The End of Company’s Response]"
    ]

    return prompt_a, prompt_b

get_prompt_medium: enforces accuracy but not length

In [ ]:
def get_prompt_medium(response_a, response_b):
    prompt_a = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        f"[The Start of Company’s Question 1 Response] {response_a} [The End of Company’s Response]"
    ]
    
    prompt_b = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        f"[The Start of Company’s Question 2 Response] {response_b} [The End of Company’s Response]"
    ]

    return prompt_a, prompt_b

get_prompt_strict: enforces length and accuracy

In [ ]:
def get_prompt_strict(response_a, response_b):
    prompt_a = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        "Preserve the word count of the original response.",
        f"[The Start of Company’s Question 1 Response] {response_a} [The End of Company’s Response]"
    ]
    
    prompt_b = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        "Preserve the word count of the original response. ",
        f"[The Start of Company’s Question 2 Response] {response_b} [The End of Company’s Response]"
    ]

    return prompt_a, prompt_b

the following takes in a file w company and reponse, greenwashes, then returns the new responses for the companies

In [ ]:
companies_file = "non_a_list_sample.csv"
input_file = "sample_dataset.csv"
output_file = "greenwashed_responses.csv"

df = pd.read_csv(input_file)
c_df = pd.read_csv(companies_file)
company_name_list = df.loc[:,"Organization"]
response_a_list = df.loc[:,Response_A_2022]
response_b_list = df.loc[:,Response_B_2022]
fields = ["Company", "Old Response A", "Old Response B", "New Response A", "New Response B"]

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    # Iterate over each company's responses and generate the API prompts
    index = 0
    while(index < len(df)):
        if(company_name_list[index] in c_df['Company'].values):
        # for company_name, response_a, response_b in zip(company_name_list, response_a_list, response_b_list):
                       # Prepare prompt for Response A

            prompt_a, prompt_b = get_prompt_strict(response_a_list[index], response_b_list[index])
            # Get the model outputs for each response
            output_a = get_model_output_initial_sentences(prompt_a, client, MODEL)
            output_b = get_model_output_initial_sentences(prompt_b, client, MODEL)
            
            line = {
                "Company": company_name_list[index], 
                "Old Response A": response_a_list[index],
                "Old Response B": response_b_list[index],
                "New Response A": output_a, 
                "New Response B": output_b,
            }
            writer.writerow(line)
            print("Company", company_name_list[index])
        index += 1
    csvfile.close()

print(f"New Responses have been written to {output_file}")

go to rating_companies and pairwise_prompting to get company ratings
note: you have to change the code slightly since the column names are different.

go to rating_companies_specific to get company ratings